In [26]:
%pip install elasticsearch PyPDF2 sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [27]:
from elasticsearch import Elasticsearch
import base64
import PyPDF2
import os
from ibm_watson_studio_lib import access_project_or_space
from sentence_transformers import SentenceTransformer


# define watsonx ai access_token and Elasticsearch endpoint and username/password here. if access_token="", notebook will use file path to load pdfs(for local usage) rather than wslib
access_token=""
access_token="p-2+ub/z14tG9QxagU+wZa4uGQ==;dvdf8yjZ06NxAPK4+gszcA==:ex4tSae4Zor3gfZ53Jznuzjc9ZioaaA/DXHq1+t1xFU64DnPgcY7lINLvrLoPD1dEdYKdVqUSjGr1mv5AQrUyETlSbOr08jNDA=="

pdf_directory = "/root/paper/"

wslib = access_project_or_space({"token": access_token})
# Connect to Elasticsearch
es = Elasticsearch(
    "https://2944028d-19a3-47ba-9ec9-dea6003f8d43.c5km1ted03t0e8geevf0.databases.appdomain.cloud:32085",
    basic_auth=("ibm_cloud_cf6ad794_9e4c_4c64_af64_0a60ad29a48c","b87062dd30645fa66cc45c44d019a2b8c3c1346184d28b2e08b4d7c8968b8425"),
    verify_certs=False,
    request_timeout=3600
)


In [28]:
index_template = {
    "index_patterns": ["pd_index*"],
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 1
    },
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "content": {"type": "text"},
            "content_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

es.indices.put_template(name="pdf_template", body=index_template)

/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '2944028d-19a3-47ba-9ec9-dea6003f8d43.c5km1ted03t0e8geevf0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True})

In [29]:
def read_and_encode_pdf(file_name):

    if access_token=="":
        file_path = os.path.join(pdf_directory, file_name)
        
        with open(file_path, "rb") as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            content = ""
            for page in pdf_reader.pages:
                content += page.extract_text()
    else:
        pdf_file = wslib.load_data(file_name)
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        content = ""
        for page in pdf_reader.pages:
            content += page.extract_text()

    encoded_pdf = base64.b64encode(pdf_file.read()).decode('ascii')
    return content, encoded_pdf


In [30]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def generate_embedding(text):
    return model.encode(text).tolist()

In [31]:
def index_pdf(file_name, index_name):

    content, encoded_pdf = read_and_encode_pdf(file_name)
    
    document = {
        "title": file_name,
        "content": content,
        "pdf_data": encoded_pdf,
        "content_vector": generate_embedding(content)  # Limit to first 512 characters
    }
    
    response = es.index(index=index_name, body=document)
    return response

In [32]:
if __name__ == "__main__":

    index_name = "pd_index"
    
    # Ensure the index exists
    if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name)

    
    if access_token=="":
        # Index PDFs
        for file_name in os.listdir(pdf_directory):
            if file_name.endswith(".pdf"):
                response = index_pdf(file_name, index_name)
                print(f"Indexed {file_name}: {response['result']}")
    else:
        #pdf_assets = ["TuringComputing.pdf", "mccolloch.logical.calculus.ideas.1943.pdf"]
        data_assets = wslib.assets.list_assets("data_asset")
        data_asset_names = [asset['name'] for asset in data_assets]
        for file_name in data_asset_names:
            if file_name.endswith('.pdf'):
                response = index_pdf(file_name, index_name)
                print(f"Indexed {file_name}: {response['result']}")

    print("Indexing complete!")

/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '2944028d-19a3-47ba-9ec9-dea6003f8d43.c5km1ted03t0e8geevf0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '2944028d-19a3-47ba-9ec9-dea6003f8d43.c5km1ted03t0e8geevf0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '2944028d-19a3-47ba-9ec9-dea6003f8d43.c5km

Indexed mccolloch.logical.calculus.ideas.1943.pdf: created


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '2944028d-19a3-47ba-9ec9-dea6003f8d43.c5km1ted03t0e8geevf0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Indexed AttentionIsAllYouNeed.pdf: created


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '2944028d-19a3-47ba-9ec9-dea6003f8d43.c5km1ted03t0e8geevf0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Indexed NIPS-2012-imagenet-classification-with-deep-convolutional-neural-networks-Paper.pdf: created


incorrect startxref pointer(1)
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '2944028d-19a3-47ba-9ec9-dea6003f8d43.c5km1ted03t0e8geevf0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Indexed TuringComputing.pdf: created
Indexed 64867.pdf: created
Indexing complete!


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '2944028d-19a3-47ba-9ec9-dea6003f8d43.c5km1ted03t0e8geevf0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [33]:
pdf_assets = wslib.assets.list_assets("data_asset")
data_asset_names = [asset['name'] for asset in pdf_assets]
print(data_asset_names)

['mccolloch.logical.calculus.ideas.1943.pdf', 'AttentionIsAllYouNeed.pdf', 'NIPS-2012-imagenet-classification-with-deep-convolutional-neural-networks-Paper.pdf', 'TuringComputing.pdf', '64867.pdf']
